In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import os
import sys

CAUSICA_FOLDER = '/home/sandor_daniel/work/2024-05-07_active_bayesian_grn/Project-BayesDAG/src/'
RESULT_DIR = '/home/sandor_daniel/work/2024-05-07_active_bayesian_grn/results/'
ROOT_DIR = '/home/sandor_daniel/work/2024-05-07_active_bayesian_grn/'
sys.path.append(ROOT_DIR)
sys.path.append(CAUSICA_FOLDER)
from causica.models.bayesdag.bayesdag_nonlinear import BayesDAGNonLinear
from causica.datasets.variables import Variables, Variable
from causica.datasets.dataset import Dataset, CausalDataset



/home/sandor_daniel/miniconda3/envs/work/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_standard(file):
    standard = pd.read_csv(file, sep='\t', header=None)
    standard.replace([f'G{i}' for i in range(10)], [f'G0{i}' for i in range(10)], inplace=True)
    standard = standard.pivot(columns=[0], index=[1], values=[2])
    np.fill_diagonal(standard.values, 0)
    standard = standard.to_numpy()
    return standard

def remove_cycles_from_true_graph(true_graph):
    G = nx.from_numpy_array(true_graph, create_using=nx.DiGraph())
    for c in nx.simple_cycles(G):
        true_graph[c[0], c[1]] = 0
    return true_graph


# Basic Bayes DAG

In [3]:
timeseries = np.loadtxt(f'gnw_example/Example_dream4_timeseries.tsv', skiprows=1)[:,1:]
timeseries_split = np.split(timeseries, range(21,210,21), axis=0)

ground_truth =load_standard(f'gnw_example/Example_goldstandard.tsv')
# ground_truth = remove_cycles_from_true_graph(ground_truth)
known_subgraph_mask = np.ones(ground_truth.shape)

graph_args = {}
graph_args['num_variables'] = timeseries.shape[1]
graph_args['exp_edges'] = None
graph_args['exp_edges_per_node'] = None
graph_args['graph_type'] = None
graph_args['seed'] = 123

n_folds = len(timeseries_split)
n_folds = 1

In [4]:
vars = Variables([Variable(f'G{i}', True, 'continuous', lower=0, upper=1)
         for i in range(1,timeseries.shape[1]+1)])

bd_graphs = []

for i in range(n_folds):

    train_data = np.vstack(timeseries_split[:i] + timeseries_split[i+1:])
    val_data = None
    test_data = timeseries_split[i]

    train_mask = np.ones(train_data.shape)
    val_mask = None
    test_mask = np.ones(test_data.shape)

    dataset = CausalDataset(train_data, 
                        train_mask, 
                        remove_cycles_from_true_graph(ground_truth), 
                        known_subgraph_mask, 
                        None, 
                        None, 
                        val_data=val_data,  
                        val_mask=val_mask,
                        test_data=test_data,
                        test_mask=test_mask,
                        graph_args=graph_args)

    bd = BayesDAGNonLinear('Model1', vars, RESULT_DIR, 'cuda:0', lambda_sparse=10, sparse_init=False)

    train_config_dict = {}
    train_config_dict['batch_size'] = 16
    train_config_dict['max_epochs'] = 4
    # train_config_dict['standardize_data_mean'] = True
    # train_config_dict['standardize_data_std'] = True
    bd.run_train(dataset, train_config_dict)
    dag_samples, is_dag = bd.get_adj_matrix(samples=16)
    bayes_dag_graphs =[dag_samples[i] for i in range(dag_samples.shape[0]) if is_dag[i]]
    bd_graphs.extend(bayes_dag_graphs)

Saving logs to /home/sandor_daniel/work/2024-05-07_active_bayesian_grn/results/train_output/summary


/home/sandor_daniel/work/2024-05-07_active_bayesian_grn/Project-BayesDAG/src/causica/models/bayesdag/bayesdag_nonlinear.py:238: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  W_adj = A_samples * vmap(self.ICGNN.get_weighted_adjacency)(params, buffers)
/home/sandor_daniel/work/2024-05-07_active_bayesian_grn/Project-BayesDAG/src/causica/models/bayesdag/bayesdag_nonlinear.py:239: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.fu

New best model found. Saving Checkpoint


OutOfMemoryError: CUDA out of memory. Tried to allocate 120.00 MiB. GPU 0 has a total capacity of 31.73 GiB of which 115.44 MiB is free. Process 335144 has 30.32 GiB memory in use. Including non-PyTorch memory, this process has 1.29 GiB memory in use. Of the allocated memory 907.15 MiB is allocated by PyTorch, and 32.85 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from utils import create_pdag, adjacency_to_edge_list, directed_shd, undirected_shd, directed_edge_f1_score, pdag_f1_score, pdag_shd, conf_matrix

d_shds = []
u_shds = []
d_f1s = []
p_f1s = []
p_shds = []

for g in bd_graphs:
    pred = adjacency_to_edge_list(g)
    
    true_graph = adjacency_to_edge_list(ground_truth)

    print(len(true_graph))
    print(len(pred))
    print(conf_matrix(true_graph, pred, len(vars)))

    d_shds.append(directed_shd(true_graph, pred))
    u_shds.append(undirected_shd(true_graph, pred))
    d_f1s.append(directed_edge_f1_score(true_graph, pred))
    true_pdag = create_pdag(remove_cycles_from_true_graph(ground_truth))
    pred_pdag = create_pdag(g)
    p_f1s.append(pdag_f1_score(true_pdag, pred_pdag))
    p_shds.append(pdag_shd(true_pdag, pred_pdag))

print(f'directed_shd: {np.mean(d_shds)}')
print(f'undirected_shd: {np.mean(u_shds)}')
print(f'directed_edge_f1_score: {np.mean(d_f1s)}')
print(f'pdag_f1_score: {np.mean(p_f1s)}')
print(f'pdag_shd: {np.mean(p_shds)}')


186
442
[[43, 143], [399, 3511]]
186
443
[[37, 149], [406, 3504]]
186
441
[[44, 142], [397, 3513]]
186
452
[[37, 149], [415, 3495]]
186
466
[[35, 151], [431, 3479]]
186
478
[[51, 135], [427, 3483]]
186
493
[[38, 148], [455, 3455]]
186
461
[[43, 143], [418, 3492]]
186
517
[[45, 141], [472, 3438]]
186
495
[[53, 133], [442, 3468]]
186
450
[[43, 143], [407, 3503]]
186
474
[[47, 139], [427, 3483]]
186
447
[[49, 137], [398, 3512]]
186
493
[[59, 127], [434, 3476]]
186
455
[[48, 138], [407, 3503]]
186
472
[[47, 139], [425, 3485]]
directed_shd: 563.5625
undirected_shd: 563.5625
directed_edge_f1_score: 0.1374563744331117
pdag_f1_score: 0.13706570269747925
pdag_shd: 587.1875


In [ ]:
print(directed_shd(true_graph, pred))
print(undirected_shd(true_graph, pred))

NameError: name 'directed_shd' is not defined

# Basic GFN